In [31]:
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
from urllib.parse import urlencode, urlparse, parse_qsl
import sys
import os
import json
from datetime import datetime, timedelta
import pandas as pd
import kaleido 
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
sys.path.append(os.path.abspath('C:/Users/Utilisateur/Documents/JEDHA/03_Data_Collection_and_Management/04_Kayak_Project/01_Weather'))
from weather import *
import boto3

In [195]:
#Here are the functions that can be used to get and save the data
df_weather = get_weather_data()
df_aggregated = get_city_ranking(df_weather,25,30,5)
save_weather_data(df_weather, df_aggregated) 

#Latest request on the 22 August 2024 - Estimated time, 10 seconds but 500 request limit per day

In [2]:
#If no additionnal scraping is needed, then this forumla is used to read the saved data
df_weather, df_aggregated = read_weather_data()



In [198]:
#Here is the script to scrap the booking data
import os
path = r'C:\Users\Utilisateur\Documents\JEDHA\03_Data_Collection_and_Management\04_Kayak_Project\02_Booking\booking\booking'
os.chdir(path)
!scrapy crawl booking -O C:/Users/Utilisateur/Documents/JEDHA/03_Data_Collection_and_Management/04_Kayak_Project/00_Cities/booking.json

#Latest request on the 22 August 2024 - Estuimated time around 6 minutes

2024-08-22 22:36:05 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: booking)
2024-08-22 22:36:06 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.0.0 (OpenSSL 3.0.14 4 Jun 2024), cryptography 42.0.5, Platform Windows-10-10.0.19045-SP0
2024-08-22 22:36:06 [scrapy.addons] INFO: Enabled addons:
[]
2024-08-22 22:36:06 [asyncio] DEBUG: Using selector: SelectSelector
2024-08-22 22:36:06 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2024-08-22 22:36:06 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.windows_events._WindowsSelectorEventLoop
2024-08-22 22:36:06 [scrapy.extensions.telnet] INFO: Telnet Password: 3782a161d64de458
2024-08-22 22:36:07 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scr

In [3]:
#Opening the from extracted from the booking website
df_booking = pd.read_json('C:/Users/Utilisateur/Documents/JEDHA/03_Data_Collection_and_Management/04_Kayak_Project/00_Cities/booking.json')

#removing the hotel without review
df_booking = df_booking[~df_booking['number_ranking'].str.contains('commentaires')]
df_booking['number_ranking'] = df_booking['number_ranking'].astype(float)

#keeping only hotels that have a large number of review
df_booking = df_booking[df_booking['number_ranking']>100]

#cleaning the distance column
df_booking['distance_value']= df_booking['distance'].str.split('\xa0').str[0].str.replace(',', '.').astype(float)
df_booking['distance_unit'] = df_booking['distance'].str.split('\xa0').str[1]
df_booking['distance_unit'] = df_booking['distance_unit'].str.split(' ').str[0]
df_booking['distance_m'] = df_booking.apply(lambda row: row['distance_value'] * 1000 if row['distance_unit'] == 'km' else row['distance_value'], axis=1)
del df_booking['distance']

#Keeping only hotel with rank above 7.5. 
df_booking['rank'] = df_booking['rank'].str.replace(',','.').astype(float)
df_booking = df_booking[df_booking['rank']>7.5]

#Changing the dateformat
df_booking['checkin'] = pd.to_datetime(df_booking['checkin'])
df_booking['checkout'] = pd.to_datetime(df_booking['checkout'])

#Chaging price format
df_booking['price'] = df_booking['price'].str.replace(' ','').astype(float)

#Sorting by name and droping index
df_booking = df_booking.sort_values(by=['name','checkin'],ignore_index=True)
df_booking.reset_index(drop=True)


,city,name,rank,number_ranking,checkin,checkout,price,adress,lat,lon,description,url,distance_value,distance_unit,distance_m
0,Saintes Maries de la mer,Abrivado,8.7,959.0,2024-08-26,2024-08-27,112.0,"2, Avenue Theodore Aubanel, 13460 Les Saintes-...",43.450337,4.426224,"Situé dans le parc régional de Camargue, l'Abr...",https://www.booking.com/hotel/fr/abrivado.fr.h...,300.0,m,300.0
1,Saintes Maries de la mer,Abrivado,8.7,959.0,2024-08-27,2024-08-28,112.0,"2, Avenue Theodore Aubanel, 13460 Les Saintes-...",43.450337,4.426224,"Situé dans le parc régional de Camargue, l'Abr...",https://www.booking.com/hotel/fr/abrivado.fr.h...,250.0,m,250.0
2,Saintes Maries de la mer,Abrivado,8.7,959.0,2024-08-28,2024-08-29,170.0,"2, Avenue Theodore Aubanel, 13460 Les Saintes-...",43.450337,4.426224,"Situé dans le parc régional de Camargue, l'Abr...",https://www.booking.com/hotel/fr/abrivado.fr.h...,150.0,m,150.0
3,Cassis,Appartement Cassis vue mer d'exception,8.3,102.0,2024-08-27,2024-08-28,182.0,"20 AVENUE MARECHAL FOCH, 13260 Cassis, France",43.218756,5.544759,Offrant une terrasse bien exposée et une vue s...,https://www.booking.com/hotel/fr/appartement-s...,300.0,m,300.0
4,Cassis,Appartement le Quai du Port Cassis parking clim,9.3,124.0,2024-08-24,2024-08-25,396.0,"24 Rue du Jeune Anacharsis, 13260 Cassis, France",43.214836,5.536076,"Situé à Cassis, l'Appartement le Quai du Port ...",https://www.booking.com/hotel/fr/appartement-l...,1.4,km,1400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Cassis,appartement du pêcheur à Cassis à 10 mètres du...,8.3,246.0,2024-08-28,2024-08-29,343.0,"6 Rue Bremond, 13260 Cassis, France",43.215340,5.536858,"Situé à Cassis, à 400 mètres de la Grande Mer ...",https://www.booking.com/hotel/fr/appartement-d...,200.0,m,200.0
224,Aigues Mortes,Ô 36 Rempart Sud B&B,9.1,448.0,2024-08-25,2024-08-26,132.0,"36 rue Victor Hugo, 30220 Aigues-Mortes, France",43.565764,4.188757,"Offrant une vue sur la ville, l'Ô 36 Rempart S...",https://www.booking.com/hotel/fr/o-36-rempart-...,1.2,km,1200.0
225,Aigues Mortes,Ô 36 Rempart Sud B&B,9.1,448.0,2024-08-26,2024-08-27,132.0,"36 rue Victor Hugo, 30220 Aigues-Mortes, France",43.565764,4.188757,"Offrant une vue sur la ville, l'Ô 36 Rempart S...",https://www.booking.com/hotel/fr/o-36-rempart-...,100.0,m,100.0
226,Aigues Mortes,Ô 36 Rempart Sud B&B,9.1,448.0,2024-08-27,2024-08-28,132.0,"36 rue Victor Hugo, 30220 Aigues-Mortes, France",43.565764,4.188757,"Offrant une vue sur la ville, l'Ô 36 Rempart S...",https://www.booking.com/hotel/fr/o-36-rempart-...,0.8,km,800.0


In [4]:
#Here we are looking for hotel in the scope which have 2 consecutive nights available
df_booking['scope'] = 0
for i in range(0,len(df_booking)-1):
    if (df_booking.loc[i,'name']==df_booking.loc[i+1,'name']) and (df_booking.loc[i+1,'checkin']==df_booking.loc[i,'checkin'] + pd.Timedelta(days=1)):
        df_booking.loc[i,'scope']=1

#Here is calculated the number of hotel in the scope available for 2 consecurive nights depending on the 
df_booking_available = df_booking[df_booking['scope']==1]
df_booking_available= df_booking_available.reset_index(drop=True)
df_booking_available.groupby(['city','checkin'])['scope'].count().reset_index()



,city,checkin,scope
0,Aigues Mortes,2024-08-23,3
1,Aigues Mortes,2024-08-24,3
2,Aigues Mortes,2024-08-25,9
3,Aigues Mortes,2024-08-26,8
4,Aigues Mortes,2024-08-27,9
5,Bormes les Mimosas,2024-08-23,8
6,Bormes les Mimosas,2024-08-24,4
7,Bormes les Mimosas,2024-08-25,5
8,Bormes les Mimosas,2024-08-26,5
9,Bormes les Mimosas,2024-08-27,4


In [38]:
#For all top cities, lets draw the maps of the available hotels in the next week with 2 consecutives dates available
top_cities = df_aggregated[:5]['city'].unique()
for city in top_cities:
    df_temp = df_booking_available[df_booking_available['city'] == city].copy()

    
    fig = px.scatter_mapbox(
        df_temp,
        lat='lat',
        lon='lon',
        size='price',
        color='rank',
        color_continuous_scale=px.colors.cyclical.IceFire, 
        size_max=15, 
        zoom=10,
        title=f'Hotels available in {city}',
        hover_name="name"
    )
    
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(
    width=800,  # Width in pixels
    height=600  # Height in pixels
)
    fig.show()

In [6]:
#Let's get the coordinates of each city to create the temperature maps
path = Path('C:/Users/Utilisateur/Documents/JEDHA/03_Data_Collection_and_Management/04_Kayak_Project/00_Cities/coordinates.csv')
df_coordinates = pd.read_csv(path)

df_aggregated_with_coordinate = pd.merge(df_aggregated,df_coordinates,on='city',how='left')

df_aggregated_with_coordinate['tier'] = 'Not Selected'
df_aggregated_with_coordinate.loc[df_aggregated_with_coordinate['id'] <= 5, 'tier'] = 'Top City'


In [37]:
fig = px.scatter_mapbox(
    df_aggregated_with_coordinate,
    lat='lat',
    lon='lon',
    size='tmax_mean',
    color='tmax_mean',
    color_continuous_scale=px.colors.cyclical.IceFire, 
    size_max=15, 
    zoom=4,
    title=f'Average max temperature for the next 7 days per city',
    hover_name="city"
)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(
    width=800,  # Width in pixels
    height=600  # Height in pixels
)
fig.show()


In [ ]:
session = boto3.Session(aws_access_key_id="", 
                        aws_secret_access_key="")
s3 = session.resource("s3")
bucket_name = 'louislepogamjedhabucketkayak'
bucket = s3.Bucket(bucket_name) 

In [13]:
#Let's now save the file in a S3 bucket
bucket.upload_file('C:/Users/Utilisateur/Documents/JEDHA/03_Data_Collection_and_Management/04_Kayak_Project/00_Cities/weather.csv','weather.csv')
bucket.upload_file('C:/Users/Utilisateur/Documents/JEDHA/03_Data_Collection_and_Management/04_Kayak_Project/00_Cities/weather_aggregated.csv','weather_aggregated.csv')
bucket.upload_file('C:/Users/Utilisateur/Documents/JEDHA/03_Data_Collection_and_Management/04_Kayak_Project/00_Cities/booking.json','booking.json')

In [39]:
#Add all the images in a folder in the S3
path = 'C:/Users/Utilisateur/Documents/JEDHA/03_Data_Collection_and_Management/04_Kayak_Project/00_Cities/image'
for file in os.listdir(path):
    temp_path = os.path.join(path, file)
    bucket.upload_file(temp_path,'image/'+file)
